In [1]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

In [2]:
uri = "bolt://34.128.78.140:7687"
username = "neo4j"
password = "unej1234"

driver = GraphDatabase.driver(uri, auth=(username, password))

In [5]:
def extract_node_properties():
    with driver.session() as session:
        # Cypher query to fetch node properties
        query = """
        MATCH (n)
        RETURN n.Vector AS vector, n.label AS label, labels(n) AS kelas, n.abstract AS keterangan
        """
        result = session.run(query)
        # Extract properties and store in DataFrame
        df = pd.DataFrame([record.values() for record in result], columns=result.keys())
        return df


In [10]:
node_df = extract_node_properties()
cleaned_kelas = node_df['kelas'].apply(lambda entry: [item for item in entry if item not in ["Resource", "NamedIndividual"]])
node_df['kelas'] = cleaned_kelas
node_df

,vector,label,kelas,keterangan
0,"[-0.48007139563560486, 0.6460448503494263, -0....",Metalaxyl,[Fungisida],Metalaxyl adalah sejenis fungisida yang diguna...
1,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],"Penyakit gosong bulir padi, juga dikenal sebag..."
2,"[0.022821197286248207, 0.3413693904876709, -0....",Acidovorax avenae subsp. avenae,[PatogenPadi],Acidovorax avenae subsp. avenae adalah bakteri...
3,"[-0.0965781956911087, 0.03939099982380867, -0....",Tanaman layu,[Gejala],Gejala layu pada tanaman padi adalah tanda yan...
4,"[0.39404162764549255, -0.3328622579574585, 0.4...",Bibit mati,[Gejala],Gejala kematian bibit padi dapat terlihat dala...
...,...,...,...,...
100,"[0.6989031434059143, -0.3856666088104248, 0.55...",Bacillus thuringiensis var. israelensis,[Pestisida],Bacillus thuringiensis var. israelensis (Bti) ...
101,"[0.4118303656578064, -0.5445472598075867, -0.1...",Belalang Sawah,[HamaPadi],"Belalang sawah, atau Nilaparvata lugens, adala..."
102,"[-0.4613795280456543, 0.18603742122650146, 0.3...",Bakteri Garis Daun,[PenyakitPadi],Penyakit bakteri garis daun pada padi disebabk...
103,"[-0.24614253640174866, 0.1480824500322342, -0....",Fusarium spp.,[PatogenPadi],Fusarium spp. adalah kelompok jamur patogen ya...


In [17]:
def extract_node_properties_and_relationships():
    with driver.session() as session:
        # Cypher query to fetch node properties and relationships
        query = """
        MATCH (n)-[r]->(m)
        RETURN n.Vector AS source_vector, n.label AS source_label, labels(n) AS source_class, 
               type(r) AS relationship_type,
               m.Vector AS target_vector, m.label AS target_label, labels(m) AS target_class
        """
        result = session.run(query)
        # Extract properties and relationships and store in DataFrame
        df = pd.DataFrame([record.values() for record in result], columns=result.keys())
        # Clean labels
        df['source_class'] = df['source_class'].apply(lambda entry: [item for item in entry if item not in ["Resource", "NamedIndividual"]])
        df['target_class'] = df['target_class'].apply(lambda entry: [item for item in entry if item not in ["Resource", "NamedIndividual"]])
        return df

# Call the function to extract node properties and relationships
Rice_KG_df = extract_node_properties_and_relationships()

# Print the DataFrame
Rice_KG_df


,source_vector,source_label,source_class,relationship_type,target_vector,target_label,target_class
0,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],diberikanFungisida,"[-0.0959160327911377, 0.14652732014656067, 0.0...",Pyraclostrobin,[Fungisida]
1,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],memilikiGejala,"[0.10879369080066681, 0.07831495255231857, -0....",Bulir mengalami kerusakan,[Gejala]
2,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],memilikiGejala,"[0.6842259168624878, -0.09251340478658676, 0.0...",Bulir pecah,[Gejala]
3,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],terkenaPatogen,"[0.24190634489059448, 0.04473293945193291, 0.0...",Tilletia barclayana,[PatogenPadi]
4,"[0.26170042157173157, 0.07159162312746048, 0.0...",Gosong bulir,[PenyakitPadi],memilikiGejala,"[0.2921145260334015, -0.051839686930179596, -0...",Bulir terdapat bercak,[Gejala]
...,...,...,...,...,...,...,...
142,"[-0.4613795280456543, 0.18603742122650146, 0.3...",Bakteri Garis Daun,[PenyakitPadi],memilikiGejala,"[-0.3528200387954712, -0.11492333561182022, 0....",Daun mengering,[Gejala]
143,"[-0.4613795280456543, 0.18603742122650146, 0.3...",Bakteri Garis Daun,[PenyakitPadi],diberikanBakterisida,"[-0.43456384539604187, 0.053754474967718124, 0...",Fosfit,[Bakterisida]
144,"[-0.4613795280456543, 0.18603742122650146, 0.3...",Bakteri Garis Daun,[PenyakitPadi],diberikanBakterisida,"[-0.5201900601387024, -0.21731899678707123, -0...",Streptomisin,[Bakterisida]
145,"[-0.4613795280456543, 0.18603742122650146, 0.3...",Bakteri Garis Daun,[PenyakitPadi],memilikiGejala,"[-0.8089390993118286, -0.32538989186286926, 0....",Malai mengalami kerusakan,[Gejala]
